In [1]:
from rasa_core_sdk.forms import FormAction
from rasa_core_sdk import Tracker, ActionExecutionRejection
from rasa_core_sdk.executor import CollectingDispatcher
from rasa_core_sdk.events import SlotSet, Form

from actions.procs.restaurant_form import RestaurantForm
form = RestaurantForm()

tracker = Tracker('default', {'requested_slot': 'some_slot'},
                  {'entities': [{'entity': 'some_slot',
                                 'value': 'some_value'},
                                {'entity': 'some_other_slot',
                                 'value': 'some_other_value'}]},
                  [], False, None, 
                  {'name': 'restaurant_form',
                       'validate': True, 'rejected': False},
                  'action_listen')

dispatcher=CollectingDispatcher()
domain={}
events = form._validate_if_required(dispatcher, tracker, domain)
for ev in events:
    print(str(ev))

# create temp tracker with populated slots from `validate` method
temp_tracker = tracker.copy()
for e in events:
    if e['event'] == 'slot':
        temp_tracker.slots[e["name"]] = e["value"]
next_slot_events = form.request_next_slot(dispatcher, temp_tracker,
                                                  domain)

print("☈", next_slot_events)

{'event': 'slot', 'timestamp': None, 'name': 'some_slot', 'value': 'some_value'}
☈ [{'event': 'slot', 'timestamp': None, 'name': 'requested_slot', 'value': 'cuisine'}]


In [2]:
from rasa_core_sdk.forms import FormAction
from rasa_core_sdk import Tracker, ActionExecutionRejection
from rasa_core_sdk.executor import CollectingDispatcher
from rasa_core_sdk.events import SlotSet, Form
from rasa_core.domain import Domain

import warnings
import ruamel.yaml.error
warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)
  
from actions.procs.restaurant_form import RestaurantForm
form = RestaurantForm()
DOMAIN_PATH="examples/formbot/domain.yml"

## sender_id, slots,
#                  latest_message, events, paused, followup_action,
#                  active_form, latest_action_name
tracker = Tracker('default', {'requested_slot': 'cuisine'},
                  {'entities': [{'entity': 'cuisine',
                                 'value': 'some_value'},
                                {'entity': 'some_other_slot',
                                 'value': 'some_other_value'}]},
                  [], False, None, 
                  {'name': 'restaurant_form',
                       'validate': True, 'rejected': False},
                  'action_listen')
print("req", tracker.slots, tracker.latest_message)

dispatcher=CollectingDispatcher()
domain=Domain.load(DOMAIN_PATH)
events=form.run(dispatcher, tracker, domain)
for ev in events:
    print(str(ev))
print("☈", dispatcher.messages)

req {'requested_slot': 'cuisine'} {'entities': [{'entity': 'cuisine', 'value': 'some_value'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}
{'event': 'slot', 'timestamp': None, 'name': 'cuisine', 'value': None}
{'event': 'slot', 'timestamp': None, 'name': 'requested_slot', 'value': 'cuisine'}
☈ [{'template': 'utter_wrong_cuisine'}, {'template': 'utter_ask_cuisine'}]


In [26]:
form = RestaurantForm()

tracker = Tracker('default', {'requested_slot': 'cuisine'},
                  {'entities': [{'entity': 'cuisine',
                                 'value': 'chinese'},
                                {'entity': 'some_other_slot',
                                 'value': 'some_other_value'}]},
                  [], False, None, 
                  {'name': 'restaurant_form',
                       'validate': True, 'rejected': False},
                  'action_listen')
print("✁ req", tracker.slots, tracker.latest_message)

dispatcher=CollectingDispatcher()
domain=Domain.load(DOMAIN_PATH)
events=form.run(dispatcher, tracker, domain)
for ev in events:
    print(str(ev))
print("☈", dispatcher.messages)
print(form.required_slots(tracker))

✁ req {'requested_slot': 'cuisine'} {'entities': [{'entity': 'cuisine', 'value': 'chinese'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}
{'event': 'slot', 'timestamp': None, 'name': 'cuisine', 'value': 'chinese'}
{'event': 'slot', 'timestamp': None, 'name': 'requested_slot', 'value': 'num_people'}
☈ [{'template': 'utter_ask_num_people'}]
['cuisine', 'num_people', 'outdoor_seating', 'preferences', 'feedback']


In [27]:
from utils import dump
dump(tracker.current_state())

{
  "sender_id": "default",
  "slots": {
    "requested_slot": "cuisine"
  },
  "latest_message": {
    "entities": [
      {
        "entity": "cuisine",
        "value": "chinese"
      },
      {
        "entity": "some_other_slot",
        "value": "some_other_value"
      }
    ]
  },
  "latest_event_time": null,
  "paused": false,
  "events": [],
  "active_form": {
    "name": "restaurant_form",
    "validate": true,
    "rejected": false
  },
  "latest_action_name": "action_listen"
}


In [11]:
import logging
import sys
from utils import dump

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

form = RestaurantForm()

tracker = Tracker('default', {'requested_slot': 'num_people'},
                  {'intent': {'name': 'inform',
                                  'confidence': 1.0},
                   'entities': [{'entity': 'cuisine',
                                 'value': 'chinese'},
                                {'entity': 'number',
                                 'value': '8'},
                                {'entity': 'feedback',
                                 'value': 'its good'},
                                {'entity': 'seating',
                                 'value': 'in'},
                                {'entity': 'some_other_slot',
                                 'value': 'some_other_value'}]
                    },
                  [], False, None, 
                  {'name': 'restaurant_form',
                       'validate': True, 'rejected': False},
                  'action_listen')
print("✁ req", tracker.slots, tracker.latest_message)

dispatcher=CollectingDispatcher()
domain=Domain.load(DOMAIN_PATH)
events=form.run(dispatcher, tracker, domain)
for ev in events:
    print(str(ev))
print("☈", dispatcher.messages)
print(form.required_slots(tracker))
# dump(tracker.current_state())

✁ req {'requested_slot': 'num_people'} {'intent': {'name': 'inform', 'confidence': 1.0}, 'entities': [{'entity': 'cuisine', 'value': 'chinese'}, {'entity': 'number', 'value': '8'}, {'entity': 'feedback', 'value': 'its good'}, {'entity': 'seating', 'value': 'in'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}
2018-12-24 20:29:40,888 | DEBUG : The form '{'name': 'restaurant_form', 'validate': True, 'rejected': False}' is active
2018-12-24 20:29:40,889 | DEBUG : Validating user input '{'intent': {'name': 'inform', 'confidence': 1.0}, 'entities': [{'entity': 'cuisine', 'value': 'chinese'}, {'entity': 'number', 'value': '8'}, {'entity': 'feedback', 'value': 'its good'}, {'entity': 'seating', 'value': 'in'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}'
2018-12-24 20:29:40,890 | DEBUG : Extracted 'chinese' for extra slot 'cuisine'
2018-12-24 20:29:40,891 | DEBUG : Extracted 'its good' for extra slot 'feedback'
2018-12-24 20:29:40,892 | DEBUG : Trying to extract

In [12]:
import logging
import sys
from utils import dump

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.DEBUG, stream=sys.stdout)

form = RestaurantForm()

tracker = Tracker('default', {'requested_slot': 'outdoor_seating'},
                  {'intent': {'name': 'inform',
                                  'confidence': 1.0},
                   'entities': [{'entity': 'cuisine',
                                 'value': 'chinese'},
                                {'entity': 'number',
                                 'value': '8'},
                                {'entity': 'feedback',
                                 'value': 'its good'},
                                {'entity': 'seating',
                                 'value': 'in'},
                                {'entity': 'some_other_slot',
                                 'value': 'some_other_value'}]
                    },
                  [], False, None, 
                  {'name': 'restaurant_form',
                       'validate': True, 'rejected': False},
                  'action_listen')
print("✁ req", tracker.slots, tracker.latest_message)

dispatcher=CollectingDispatcher()
domain=Domain.load(DOMAIN_PATH)
events=form.run(dispatcher, tracker, domain)
for ev in events:
    print(str(ev))
print("☈", dispatcher.messages)
print(form.required_slots(tracker))

✁ req {'requested_slot': 'outdoor_seating'} {'intent': {'name': 'inform', 'confidence': 1.0}, 'entities': [{'entity': 'cuisine', 'value': 'chinese'}, {'entity': 'number', 'value': '8'}, {'entity': 'feedback', 'value': 'its good'}, {'entity': 'seating', 'value': 'in'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}
2018-12-24 20:34:27,389 | DEBUG : The form '{'name': 'restaurant_form', 'validate': True, 'rejected': False}' is active
2018-12-24 20:34:27,390 | DEBUG : Validating user input '{'intent': {'name': 'inform', 'confidence': 1.0}, 'entities': [{'entity': 'cuisine', 'value': 'chinese'}, {'entity': 'number', 'value': '8'}, {'entity': 'feedback', 'value': 'its good'}, {'entity': 'seating', 'value': 'in'}, {'entity': 'some_other_slot', 'value': 'some_other_value'}]}'
2018-12-24 20:34:27,392 | DEBUG : Extracted 'chinese' for extra slot 'cuisine'
2018-12-24 20:34:27,393 | DEBUG : Extracted 'its good' for extra slot 'feedback'
2018-12-24 20:34:27,394 | DEBUG : Trying to ex

In [6]:
class CustomFormAction(FormAction):
    def slot_mappings(self):
        return {"some_slot": self.from_entity(entity="some_entity",
                                              intent="some_intent")}
form = CustomFormAction()

tracker = Tracker('default', {'requested_slot': 'some_slot'},
                  {'intent': {'name': 'some_intent', 'confidence': 1.0},
                   'entities': [{'entity': 'some_entity',
                                 'value': 'some_value'}]},
                  [], False, None, {}, 'action_listen')

slot_values = form.extract_requested_slot(CollectingDispatcher(),
                                          tracker, {})
# check that the value was extracted for correct intent
assert slot_values == {'some_slot': 'some_value'}

2018-12-24 20:24:19,166 | DEBUG : Trying to extract requested slot 'some_slot' ...
2018-12-24 20:24:19,168 | DEBUG : Got mapping '{'type': 'from_entity', 'entity': 'some_entity', 'intent': ['some_intent'], 'not_intent': []}'
2018-12-24 20:24:19,170 | DEBUG : Successfully extracted 'some_value' for requested slot 'some_slot'
